In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

# Hypothetical monthly sales data
data = {
    'Month': pd.date_range(start='2020-01-01', periods=60, freq='M'),
    'Sales': np.random.lognormal(mean=2, sigma=1, size=60).cumsum()  # Log-normal growth
}
df = pd.DataFrame(data)

# Step 1: Plot the time series
plt.figure(figsize=(10, 6))
plt.plot(df['Month'], df['Sales'], label='Sales')
plt.title('Monthly Sales Data')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.legend()
plt.show()

# Step 2: Decompose the time series
result = seasonal_decompose(df['Sales'], model='additive', period=12)  # Specify the period as 12 for monthly data
result.plot()
plt.show()

# Step 3: Log transformation
df['Log_Sales'] = np.log(df['Sales'])

# Step 4: Differencing
df['D_Sales'] = df['Sales'].diff().dropna()

# Step 5: ADF Test
def adf_test(series):
    result = adfuller(series, autolag='AIC')
    output = f"ADF Statistic: {result[0]}, p-value: {result[1]}"
    for key, value in result[4].items():
        output += f"\nCrit value at {key}% confidence level: {value}"
    print(output)

print("ADF Test on Original Sales Data:")
adf_test(df['Sales'])

print("\nADF Test on Log-transformed and Differenced Data:")
adf_test(df['Log_Sales'])

# Step 6: ACF and PACF Plots
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(df['Log_Sales'], ax=plt.gca())
plot_pacf(df['Log_Sales'], ax=plt.gca())
plt.show()

# Step 7: Model Selection
# Based on the ACF and PACF plots, you can select an appropriate ARIMA model or its variants.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf

# Generate an ARIMA(1, 1, 2) process for demonstration
np.random.seed(42)  # For reproducibility
ar_params = [1.5]
ma_params = [-0.8, -0.3]
n_obs = 250
arma_process = ArmaProcess(ar=ar_params, ma=ma_params)
x = arma_process.generate_sample(nsample=n_obs)

# Plot the time series
plt.figure(figsize=(10, 6))
plt.plot(x, label='ARIMA(1, 1, 2) Process')
plt.title('Time Series Plot')
plt.xlabel('Observation')
plt.ylabel('Value')
plt.legend()
plt.show()

# Generate and plot the ACF and PACF
acf_result = acf(x, nlags=20)
pacf_result = pacf(x, nlags=20, method='ywm')

# Plot the ACF
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.plot(acf_result)
plt.title('ACF Plot')
plt.xlabel('Lag')
plt.ylabel('ACF')

# Plot the PACF
plt.subplot(122)
plt.plot(pacf_result)
plt.title('PACF Plot')
plt.xlabel('Lag')
plt.ylabel('PACF')
plt.tight_layout()
plt.show()

# Interpretation of plots:
# - The PACF cuts off after the first lag, suggesting an AR(1) process.
# - The ACF shows significant correlation up to the second lag, suggesting an MA(2) process.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Hypothetical quarterly economic data with a seasonal pattern
np.random.seed(42)
period = 4  # Quarterly data
n_observations = 80  # 20 years of quarterly data

# Create a seasonal pattern and add random shocks
# Ensure that the data does not contain zeros or negative values
seasonal_effect = np.sin(np.arange(n_observations) / period * 2. * np.pi).cumsum()
random_shock = np.random.normal(scale=0.5, size=n_observations) + 0.1  # Added 0.1 to avoid negatives
data = seasonal_effect + random_shock

# Create a DataFrame
df = pd.DataFrame(data, index=pd.date_range('2000-01-01', periods=n_observations, freq='Q'), columns=['Economic Indicator'])

# Plot the time series
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['Economic Indicator'], label='Economic Indicator')
plt.title('Quarterly Economic Indicator with Seasonal Pattern')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

# Seasonal Decompose the time series using 'additive' model
result = seasonal_decompose(df['Economic Indicator'], model='additive', period=4)
result.plot()
plt.show()

# The rest of the code for differencing, ACF/PACF plotting, and ADF testing remains the same.
# ...

In [ ]:
# Step 3: First Seasonal Differencing
df['Seasonal_Dif_First'] = df['Economic Indicator'].diff(periods=period).dropna()

# Step 4: Plot ACF and PACF of the first seasonal difference
plt.figure(figsize=(12, 5))
plot_acf(df['Seasonal_Dif_First'], ax=plt.subplot(121))
plot_pacf(df['Seasonal_Dif_First'], ax=plt.subplot(122))
plt.tight_layout()
plt.show()

# Step 5: ADF Test on the first seasonal difference
def adf_test(series, title='ADF Test'):
    result = adfuller(series.dropna(), autolag='AIC')
    print(f"{title} Statistic: {result[0]}, p-value: {result[1]}")
    for lag in result[4].keys():
        print(f"Critical Value ({lag}%): {result[4][lag]}")

print("ADF Test after first seasonal difference:")
adf_test(df['Seasonal_Dif_First'], 'First Seasonal Difference ADF Test')

# Check if second seasonal difference is needed by examining the ACF and PACF plots
# If the plots show significant seasonal patterns, apply a second seasonal difference
if np.any(acf(df['Seasonal_Dif_First'], nlags=5*period)[1] > 1.96/np.sqrt(len(df))):
    print("Second seasonal difference is needed.")
    df['Seasonal_Dif_Second'] = df['Seasonal_Dif_First'].diff(periods=period).dropna()

    # Plot ACF and PACF of the second seasonal difference
    plt.figure(figsize=(12, 5))
    plot_acf(df['Seasonal_Dif_Second'], ax=plt.subplot(121))
    plot_pacf(df['Seasonal_Dif_Second'], ax=plt.subplot(122))
    plt.tight_layout()
    plt.show()

    # ADF Test on the second seasonal difference
    print("\nADF Test after second seasonal difference:")
    adf_test(df['Seasonal_Dif_Second'], 'Second Seasonal Difference ADF Test')
else:
    print("Second seasonal difference is not needed.")